In [1]:
import numpy as np
import cv2
import random
from matplotlib import pyplot as plt

import dsbhelper as dsb
import nuclei
import model as modellib
import dsbhelper

/Users/farrar/py3/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
TRAIN_DATA = 'stage1_train'

In [3]:
# configure the trainer for inference
config = nuclei.InferenceConfig()

In [4]:
# set up the dataset and load validate data
dataset_val = nuclei.NucleiDataset()
dataset_val.load_data(TRAIN_DATA, mode='validate', filter_ids=dsb.validation_set)
dataset_val.prepare()

In [5]:
# load the model
local_model_folder = 'model'
model = modellib.MaskRCNN(mode="inference", config=config, model_dir=local_model_folder)


In [6]:
# load weights
#model.load_weights('./model/mask_rcnn.h5', by_name=True)

gcs_weights_fname = 'gs://mwpdsb/mask_rcnn/models/model_b12eb9/mask_rcnn.h5'
model.load_weights(dsbhelper.get_pretrained_weights(pretrained='last',
                                                    gcs_weights_fname=gcs_weights_fname), by_name=True)

In [7]:
# pick a specimen
image_id = random.choice(dataset_val.image_ids)
#image_id = 25
image_data = dataset_val.get_info(image_id)
print(image_data['specimen_id'])

92f31f591929a30e4309ab75185c96ff4314ce0a7ead2ed2c2171897ad1da0c7


In [8]:
#for image_id in dataset_val.image_ids:
#    image_data = dataset_val.get_info(image_id)
#    print('{} -> {}'.format(image_id, image_data['specimen_id']))

In [9]:
# get ground truth data
original_image, _, \
class_ids, \
rois,\
gt_masks = modellib.load_image_gt(dataset_val, config, image_id, use_mini_mask=False)

/Users/farrar/py3/lib/python3.5/site-packages/scipy/ndimage/interpolation.py:616: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


In [10]:
print(gt_masks.shape[-1])

10


In [11]:
print(gt_masks.shape)
nrows = int(gt_masks.shape[-1]/4) + 1
fig, axarr = plt.subplots(nrows=nrows, ncols=4, figsize=(10,10))

i = 0
for row in range(nrows):
    for col in range(4):
        if i < gt_masks.shape[-1]:
            axarr[row, col].imshow(gt_masks[:, :, i])
        i += 1

(512, 512, 10)


In [12]:
# run inference on the same image
p = model.detect([original_image], verbose=1)
prediction = p[0]
print(prediction['masks'].shape)
print(prediction['rois'].shape)
print(prediction['scores'].shape)
print(prediction['class_ids'].shape)

Processing 1 images
image                    shape: (512, 512, 3)         min:    0.00000  max:  232.00000
molded_images            shape: (1, 512, 512, 3)      min:  -46.82168  max:  189.82254
image_metas              shape: (1, 10)               min:    0.00000  max:  512.00000
(512, 512, 10)
(10, 4)
(10,)
(10,)


In [13]:
masks = prediction['masks']
print(masks.shape)

(512, 512, 10)


In [14]:
nrows = int(masks.shape[-1]/4) + 1
if nrows > 4:
    nrows = 4
fig, axarr = plt.subplots(nrows=nrows, ncols=4, figsize=(10,10))

i = 0
for row in range(nrows):
    for col in range(4):
        if i < masks.shape[-1]:
            axarr[row, col].imshow(masks[:, :, i])
        i += 1

In [15]:
print(gt_masks.shape)
print(masks.shape)
ap = dsb.compute_ap(gt_masks, masks)
print(ap)

(512, 512, 10)
(512, 512, 10)
0.2669237195552985


In [16]:
masks = prediction['masks']
print(masks.shape)

(512, 512, 10)


In [17]:
new_masks = []
for i in range(masks.shape[-1]):
    rgb = cv2.cvtColor(masks[:, :, i], cv2.COLOR_GRAY2RGB)
    gray = cv2.cvtColor(rgb, cv2.COLOR_RGB2GRAY)
    _, contours, _ = cv2.findContours(gray.astype(np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    for j, contour in enumerate(contours):
        area = cv2.contourArea(contour)
        print('masks[{}] -> contour[{}] - > area={}'.format(i, j, area))
        if area >= 80:
            new_masks.append(masks[:, :, i])

masks[0] -> contour[0] - > area=909.5
masks[1] -> contour[0] - > area=755.0
masks[2] -> contour[0] - > area=175.0
masks[3] -> contour[0] - > area=649.0
masks[4] -> contour[0] - > area=876.5
masks[5] -> contour[0] - > area=627.5
masks[6] -> contour[0] - > area=252.0
masks[7] -> contour[0] - > area=475.5
masks[8] -> contour[0] - > area=221.0
masks[9] -> contour[0] - > area=146.5


In [18]:
new_masks = np.asarray(new_masks)
print(new_masks.shape)
plt.imshow(new_masks[0])

(10, 512, 512)


In [19]:
new_masks = np.moveaxis(new_masks, 0, 2)
print(new_masks.shape)
plt.imshow(new_masks[:, :, 0])

(512, 512, 10)


In [20]:
new_ap = dsb.compute_ap(gt_masks, new_masks)
print(new_ap)

0.2669237195552985


In [24]:
print(prediction['masks'].shape)
clean_masks, clean_rois, clean_scores, clean_class_ids = dsb.drop_small_masks(prediction['masks'], 
                                                                              prediction['rois'],
                                                                              prediction['class_ids'],
                                                                              prediction['scores'])
print (clean_masks)
#plt.imshow(clean_masks[:,:,0])

(512, 512, 10)
[]


In [ ]:
nrows = int(clean_masks.shape[-1]/4) + 1
if nrows > 4:
    nrows = 4
fig, axarr = plt.subplots(nrows=nrows, ncols=4, figsize=(10,10))
i = 0

for row in range(nrows):
    for col in range(4):
        if i < clean_masks.shape[-1]:
            axarr[row, col].imshow(clean_masks[:, : , i])
        i += 1

In [ ]:
new_ap = dsb.compute_ap(gt_masks, clean_masks)
print (new_ap)

In [ ]:
def open_masks(masks):
    dilated_masks = np.zeros_like(masks)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(9,9)) #0.694064

    for i in range(masks.shape[2]):   
        dilated_masks[:, :, i] = cv2.morphologyEx(masks[:, :, i], cv2.MORPH_OPEN, kernel)

    return dilated_masks

In [ ]:
#image_id = random.choice(dataset_val.image_ids)
image_id = 25
image_data = dataset_val.get_info(image_id)
print(image_data['specimen_id'])

In [ ]:
# get ground truth data
original_image, _, \
class_ids, \
rois,\
gt_masks = modellib.load_image_gt(dataset_val, config, image_id, use_mini_mask=False)

In [ ]:
# run inference on the same image
p = model.detect([original_image], verbose=1)
prediction = p[0]
print(prediction['masks'].shape)
print(prediction['rois'].shape)
print(prediction['scores'].shape)
print(prediction['class_ids'].shape)

In [ ]:
masks = prediction['masks']
dilated_masks = open_masks(masks)
ap = dsb.compute_ap(gt_masks, masks)
new_ap = dsb.compute_ap(gt_masks, dilated_masks)
print ('was -> {}'.format(ap))
print ('now -> {}'.format(new_ap))

In [ ]:
for i, image_id in enumerate(dsb.validation_set):
    image_id = i
    image_data = dataset_val.get_info(image_id)
    print(image_data['specimen_id'])
    
    # get ground truth data
    original_image, _, \
    class_ids, \
    rois,\
    gt_masks = modellib.load_image_gt(dataset_val, config, image_id, use_mini_mask=False)
    
    # run inference on the same image
    p = model.detect([original_image], verbose=0)
    prediction = p[0]

    masks = prediction['masks']
    dilated_masks = open_masks(masks)
    ap = dsb.compute_ap(gt_masks, masks)
    new_ap = dsb.compute_ap(gt_masks, dilated_masks)
    print ('was -> {}'.format(ap))
    print ('now -> {}'.format(new_ap))